In [1]:
import sys
sys.path
# sys.path.append('devel/lib/python3/dist-packages/theconstruct_msgs/msg/')
# sys.path.append('/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/')
# sys.path.append('/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib/python3/dist-packages/openai_ros/msg')

['/home/nils/Documents/moveo_ws',
 '',
 '/home/nils/Documents/moveo_ws/devel/lib/python3/dist-packages',
 '/opt/ros/noetic/lib/python3/dist-packages',
 '/home/nils/anaconda3/lib/python38.zip',
 '/home/nils/anaconda3/lib/python3.8',
 '/home/nils/anaconda3/lib/python3.8/lib-dynload',
 '/home/nils/.local/lib/python3.8/site-packages',
 '/home/nils/anaconda3/lib/python3.8/site-packages',
 '/home/nils/anaconda3/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/nils/.local/lib/python3.8/site-packages/IPython/extensions',
 '/home/nils/.ipython']

In [2]:
import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile
import PIL.Image

import tensorflow as tf
from tf_agents.environments import suite_gym
from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.environments import suite_pybullet
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.policies import tf_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils
import rospy

from src.moveo_training.src import moveo_ik_env
from src.moveo_training.src import moveo_push
from src.moveo_training.src import moveo_pick
tempdir = tempfile.gettempdir()

2021-09-07 14:09:10.214537: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nils/Documents/moveo_ws/devel/lib:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu
2021-09-07 14:09:10.214574: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Failed to import pyassimp, see https://github.com/ros-planning/moveit/issues/86 for more info


In [3]:
num_iterations =1000# @param {type:"integer"}

initial_collect_steps = 100 # @param {type:"integer"}
collect_steps_per_iteration = 100 # @param {type:"integer"}
replay_buffer_capacity = 10000 # @param {type:"integer"}

batch_size = 256 # @param {type:"integer"}

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 100 # @param {type:"integer"}

num_eval_episodes = 5 # @param {type:"integer"}
eval_interval = 50 # @param {type:"integer"}

policy_save_interval = 100 # @param {type:"integer"}

In [4]:
rospy.init_node("train_moveo")

In [5]:
# env_name = 'MoveoIK-v0' # @param {type:"string"}
# env_name = 'MoveoPush-v0' # @param {type:"string"}
env_name = 'MoveoPick-v0' # @param {type:"string"}

In [6]:
env = suite_gym.load(env_name)

Entered Push Env
Entered Gazebo Env
False


[ERROR] [1631016554.696609, 157377.607000]: NOT Initialising Simulation Physics Parameters
[WARN] [1631016554.707513, 157377.635000]: Start Init ControllersConnection
[WARN] [1631016554.708822, 157377.635000]: END Init ControllersConnection
/home/nils/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [7]:
env.reset()

Entered reset


TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([6.7612398e-01, 1.9698759e-12, 6.7612398e-01], dtype=float32),
 'reward': array(0., dtype=float32),
 'step_type': array(0, dtype=int32)})

In [8]:
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Action Spec:')
print(env.action_spec())


Observation Spec:
BoundedArraySpec(shape=(3,), dtype=dtype('float32'), name='observation', minimum=[0.  0.  0.3], maximum=[3. 1. 1.])
Action Spec:
BoundedArraySpec(shape=(6,), dtype=dtype('float32'), name='action', minimum=-3.140000104904175, maximum=3.140000104904175)


In [9]:
collect_env = suite_gym.load(env_name)
eval_env = suite_gym.load(env_name)

Entered Push Env
Entered Gazebo Env
False


[ERROR] [1631016556.085258, 157384.028000]: NOT Initialising Simulation Physics Parameters
[WARN] [1631016556.097991, 157384.095000]: Start Init ControllersConnection
[WARN] [1631016556.099201, 157384.096000]: END Init ControllersConnection


Entered Push Env
Entered Gazebo Env
False


[ERROR] [1631016556.917588, 157387.970000]: NOT Initialising Simulation Physics Parameters
[WARN] [1631016556.928714, 157388.016000]: Start Init ControllersConnection
[WARN] [1631016556.929942, 157388.016000]: END Init ControllersConnection


In [10]:
use_gpu = False

In [11]:
strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

In [12]:
observation_spec, action_spec, time_step_spec = (
      spec_utils.get_tensor_specs(collect_env))

In [13]:
with strategy.scope():
  critic_net = critic_network.CriticNetwork(
        (observation_spec, action_spec),
        observation_fc_layer_params=None,
        action_fc_layer_params=None,
        joint_fc_layer_params=critic_joint_fc_layer_params,
        kernel_initializer='glorot_uniform',
        last_kernel_initializer='glorot_uniform')

In [14]:
with strategy.scope():
  actor_net = actor_distribution_network.ActorDistributionNetwork(
      observation_spec,
      action_spec,
      fc_layer_params=actor_fc_layer_params,
      continuous_projection_net=(
          tanh_normal_projection_network.TanhNormalProjectionNetwork))

In [15]:
with strategy.scope():
  train_step = train_utils.create_train_step()

  tf_agent = sac_agent.SacAgent(
        time_step_spec,
        action_spec,
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=actor_learning_rate),
        critic_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=critic_learning_rate),
        alpha_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=alpha_learning_rate),
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        td_errors_loss_fn=tf.math.squared_difference,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        train_step_counter=train_step)

  tf_agent.initialize()

2021-09-07 14:09:17.241451: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-09-07 14:09:17.281421: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-07 14:09:17.281927: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 970 computeCapability: 5.2
coreClock: 1.2155GHz coreCount: 13 deviceMemorySize: 3.94GiB deviceMemoryBandwidth: 208.91GiB/s
2021-09-07 14:09:17.282114: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nils/Documents/moveo_ws/devel/lib:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu
2021-09-07 1

In [16]:
rate_limiter=reverb.rate_limiters.SampleToInsertRatio(samples_per_insert=3.0, min_size_to_sample=3, error_buffer=3.0)

In [17]:
table_name = 'uniform_table'
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1))

reverb_server = reverb.Server([table])

[reverb/cc/platform/tfrecord_checkpointer.cc:150]  Initializing TFRecordCheckpointer in /tmp/tmptke61ec3.
[reverb/cc/platform/tfrecord_checkpointer.cc:378] Loading latest checkpoint from /tmp/tmptke61ec3
[reverb/cc/platform/default/server.cc:54] Started replay server on port 24522


In [18]:
reverb_replay = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    sequence_length=2,
    table_name=table_name,
    local_server=reverb_server)

In [19]:
dataset = reverb_replay.as_dataset(
      sample_batch_size=batch_size, num_steps=2).prefetch(50)
experience_dataset_fn = lambda: dataset

In [20]:

tf_eval_policy = tf_agent.policy
eval_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_eval_policy, use_tf_function=True)



In [21]:


tf_collect_policy = tf_agent.collect_policy
collect_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_collect_policy, use_tf_function=True)


In [22]:


random_policy = random_py_policy.RandomPyPolicy(
  collect_env.time_step_spec(), collect_env.action_spec())


In [23]:


rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  reverb_replay.py_client,
  table_name,
  sequence_length=2,
  stride_length=1)



In [24]:
initial_collect_actor = actor.Actor(
  collect_env,
  random_policy,
  train_step,
  steps_per_run=initial_collect_steps,
  observers=[rb_observer])
initial_collect_actor.run()

Entered reset
Entered step
Unpause sim
Set action
Action:
[ 1.1729516  -1.6695895   0.6274204   2.4623094   0.29894164 -3.0230181 ]
-3.0230181217193604
Set grip angle
[-0.00018647245401393064, 0.00018647245401393064, -0.00018647245401393064, 0.00018647245401393064, -0.00018647245401393064, 0.00018647245401393064]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=False,done_sucess=False
Reward for getting closser
Entered step
Unpause sim
Set action
Action:
[ 0.14095943 -2.7932396   0.9557371  -2.2937915  -0.37238476  1.7309364 ]
1.7309364080429077
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.00018646815902734204, 0.00018646815902734204, -0.00018646815902734204, 0.00018646815902734204, -0.00018646815902734204, 0.00018646815902734204]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=True,done_sucess=False
Entered reset
Entered step
Unpause sim
Set action
Action:
[-2.2395253   2.459956    1.0805808   1.8436921   0.40255842 -0.04667575]
-0.046675752848386765
Error setting jo

[ WARN] [1631016562.584816811, 157411.514000000]: Fail: ABORTED: No motion plan found. No execution attempted.


Is done
>>>>>>>>>>>>>>>>done_fail=True,done_sucess=False
Entered reset
Entered step
Unpause sim
Set action
Action:
[-1.8236399  -2.2227395  -2.0633848   2.0632484  -0.96349144 -0.35785204]
-0.357852041721344


[ WARN] [1631016562.921873941, 157412.958000000]: Fail: ABORTED: No motion plan found. No execution attempted.


Set grip angle
[-0.00018616496323886622, 0.00018616496323886622, -0.00018616496323886622, 0.00018616496323886622, -0.00018616496323886622, 0.00018616496323886622]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=False,done_sucess=False
Reward for getting closser
Entered step
Unpause sim
Set action
Action:
[-2.782414    0.18313175  2.478037   -0.9415516   1.7199618  -2.9419963 ]
-2.9419963359832764
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.00018616496323886622, 0.00018616496323886622, -0.00018616496323886622, 0.00018616496323886622, -0.00018616496323886622, 0.00018616496323886622]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=True,done_sucess=False
Entered reset
Entered step
Unpause sim
Set action
Action:
[ 1.0343248  3.0335858 -2.0916166  2.5931587 -1.0682864 -1.1888474]
-1.1888474225997925
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.00018616496323886622, 0.00018616496323886622, -0.00018616496323886622, 0.00018616496323886622, -0.0

...

Set grip angle
[-0.00018616479784316908, 0.00018616479784316908, -0.00018616479784316908, 0.00018616479784316908, -0.00018616479784316908, 0.00018616479784316908]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=False,done_sucess=False
Reward for getting closser
Entered step
Unpause sim
Set action
Action:
[ 0.48904863  0.4995834  -0.08537874  1.6043582   1.376421   -1.8667169 ]
-1.8667168617248535
Set grip angle
[-0.0001861245711056725, 0.0001861245711056725, -0.0001861245711056725, 0.0001861245711056725, -0.0001861245711056725, 0.0001861245711056725]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=False,done_sucess=False
Reward for getting closser
Entered step
Unpause sim
Set action
Action:
[1.0036194  2.266615   2.996644   0.1883039  1.5856965  0.32146016]
0.32146015763282776
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.0001861245711056725, 0.0001861245711056725, -0.0001861245711056725, 0.0001861245711056725, -0.0001861245711056725, 0.0001861245711056725]
Get Obs
Is

[ WARN] [1631016599.604321153, 157574.008000000]: Fail: ABORTED: No motion plan found. No execution attempted.
[ WARN] [1631016599.789033042, 157574.737000000]: Fail: ABORTED: No motion plan found. No execution attempted.


Set grip angle
[-0.0001861649456857961, 0.0001861649456857961, -0.0001861649456857961, 0.0001861649456857961, -0.0001861649456857961, 0.0001861649456857961]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=False,done_sucess=False
Reward for getting closser
Entered step
Unpause sim
Set action
Action:
[-1.1837853   0.4291323   1.8955656  -0.6572299  -2.9715195  -0.09092826]
-0.09092825651168823
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.0001861649456857961, 0.0001861649456857961, -0.0001861649456857961, 0.0001861649456857961, -0.0001861649456857961, 0.0001861649456857961]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=True,done_sucess=False
Entered reset
Entered step
Unpause sim
Set action
Action:
[-0.29174405  0.16544299 -2.4190223   0.54098433  0.5349197  -0.24107951]
-0.24107950925827026
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.00018616494568401976, 0.00018616494568401976, -0.00018616494568401976, 0.00018616494568401976, -0.00018

[ WARN] [1631016600.373571586, 157576.886000000]: Fail: ABORTED: No motion plan found. No execution attempted.


Set grip angle
[-0.00018744300215445264, 0.00018744300215445264, -0.00018744300215445264, 0.00018744300215445264, -0.00018744300215445264, 0.00018744300215445264]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=False,done_sucess=False
Reward for getting closser
Entered step
Unpause sim
Set action
Action:
[ 1.4753441  -0.31199405 -2.597019   -1.3335948   1.9758071   3.0593634 ]
3.05936336517334
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.00018744300215356446, 0.00018744300215356446, -0.00018744300215356446, 0.00018744300215356446, -0.00018744300215356446, 0.00018744300215356446]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=True,done_sucess=False
Entered reset
Entered step
Unpause sim
Set action
Action:
[-2.2030978  1.6954947  2.928995  -0.5145822  2.0842884 -2.9992714]
-2.9992713928222656
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.00018616474905019942, 0.00018616474905019942, -0.00018616474905019942, 0.00018616474905019942, -0.0001

In [25]:
env_step_metric = py_metrics.EnvironmentSteps()
collect_actor = actor.Actor(
  collect_env,
  collect_policy,
  train_step,
  steps_per_run=1,
  metrics=actor.collect_metrics(10),
  summary_dir=os.path.join(tempdir, learner.TRAIN_DIR),
  observers=[rb_observer, env_step_metric])

Entered reset


In [26]:
eval_actor = actor.Actor(
  eval_env,
  eval_policy,
  train_step,
  episodes_per_run=num_eval_episodes,
  metrics=actor.eval_metrics(num_eval_episodes),
  summary_dir=os.path.join(tempdir, 'eval'),
)

Entered reset


In [27]:
saved_model_dir = os.path.join(tempdir, learner.POLICY_SAVED_MODEL_DIR)

In [28]:



# Triggers to save the agent's policy checkpoints.
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        tf_agent,
        train_step,
        interval=policy_save_interval),
    triggers.StepPerSecondLogTrigger(train_step, interval=1000),
]


Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it with `scale_diag` directly instead.


2021-09-07 14:10:03.862942: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [29]:

agent_learner = learner.Learner(
  tempdir,
  train_step,
  tf_agent,
  experience_dataset_fn,
  triggers=learning_triggers)

Instructions for updating:
rename to distribute_datasets_from_function


2021-09-07 14:10:04.445047: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-09-07 14:10:04.468232: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3192490000 Hz


In [30]:

def get_eval_metrics():
  print("Starte evaluierung")
  eval_actor.run()
  results = {}
  for metric in eval_actor.metrics:
    results[metric.name] = metric.result()
  print("Evaluierung beendete")
  return results

metrics = get_eval_metrics()


Starte evaluierung
Entered step
Unpause sim
Set action
Action:
[-0.6752017   2.9237332  -0.39305317  1.5295918  -2.9521177   0.82237726]
0.8223772644996643
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.00018616474904931124, 0.00018616474904931124, -0.00018616474904931124, 0.00018616474904931124, -0.00018616474904931124, 0.00018616474904931124]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=True,done_sucess=False
Entered reset
Entered step
Unpause sim
Set action
Action:
[-2.095933   -1.6866807  -0.55810124  2.623477   -0.37703595 -1.6733667 ]
-1.673366665840149
Set grip angle
[-0.00018714730451918626, 0.00018714730451918626, -0.00018714730451918626, 0.00018714730451918626, -0.00018714730451918626, 0.00018714730451918626]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=False,done_sucess=False
Reward for getting closser
Entered step
Unpause sim
Set action
Action:
[ 1.0345861  -3.0136936   0.47564355  2.3584628   1.3279288  -2.5749812 ]
-2.574981212615967
Error setting j

In [31]:


def log_eval_metrics(step, metrics):
  eval_results = (', ').join(
      '{} = {:.6f}'.format(name, result) for name, result in metrics.items())
  print('step = {0}: {1}'.format(step, eval_results))

log_eval_metrics(0, metrics)


step = 0: AverageReturn = -100.193275, AverageEpisodeLength = 1.400000


In [32]:
# Reset the train step
tf_agent.train_step_counter.assign(0)
# Evaluate the agent's policy once before training.
avg_return = get_eval_metrics()["AverageReturn"]
returns = [avg_return]
avg_episodes_length = get_eval_metrics()["AverageEpisodeLength"]
episodes_lengths = [avg_episodes_length]
for _ in range(num_iterations):
  # Training.
  collect_actor.run()
  loss_info = agent_learner.run(iterations=1)
  # Evaluating.
  step = agent_learner.train_step_numpy
  if eval_interval and step % eval_interval == 0:
    metrics = get_eval_metrics()
    log_eval_metrics(step, metrics)
   

    returns.append(metrics["AverageReturn"])
    episodes_lengths.append(metrics["AverageEpisodeLength"])
  if log_interval and step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, loss_info.loss.numpy()))


Starte evaluierung
Entered step
Unpause sim
Set action
Action:
[-0.8185997   2.496107    0.46401685  0.6268248   2.9158337  -2.3837404 ]
-2.3837404251098633
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.00018616471619647967, 0.00018616471619647967, -0.00018616471619647967, 0.00018616471619647967, -0.00018616471619647967, 0.00018616471619647967]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=True,done_sucess=False
Entered reset
Entered step
Unpause sim
Set action
Action:
[ 1.301705   1.2303219  2.8570607 -0.5620718  0.8237193  2.0854998]
2.0854997634887695
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.00018616471619647967, 0.00018616471619647967, -0.00018616471619647967, 0.00018616471619647967, -0.00018616471619647967, 0.00018616471619647967]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=True,done_sucess=False
Entered reset
Entered step
Unpause sim
Set action
Action:
[ 1.940887  -2.6310785 -2.0559077  1.4098495 -1.1800624 -1.1636521]
-1

[reverb/cc/client.cc:162] Sampler and server are owned by the same process (288808) so Table uniform_table is accessed directly without gRPC.
[reverb/cc/client.cc:162] Sampler and server are owned by the same process (288808) so Table uniform_table is accessed directly without gRPC.
[reverb/cc/client.cc:162] Sampler and server are owned by the same process (288808) so Table uniform_table is accessed directly without gRPC.
[reverb/cc/client.cc:162] Sampler and server are owned by the same process (288808) so Table uniform_table is accessed directly without gRPC.
[reverb/cc/client.cc:162] Sampler and server are owned by the same process (288808) so Table uniform_table is accessed directly without gRPC.
[reverb/cc/client.cc:162] Sampler and server are owned by the same process (288808) so Table uniform_table is accessed directly without gRPC.


Entered reset
Entered step
Unpause sim
Set action
Action:
[-2.0810058  2.5582652 -2.043267  -3.0673802 -2.5187795 -0.659946 ]
-0.6599460244178772
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.00018616471619647967, 0.00018616471619647967, -0.00018616471619647967, 0.00018616471619647967, -0.00018616471619647967, 0.00018616471619647967]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=True,done_sucess=False
Entered reset
Entered step
Unpause sim
Set action
Action:
[ 1.4209384   0.40455908  1.5229048  -0.2866169   0.30102444 -2.0413935 ]
-2.041393518447876
Set grip angle
[-0.0001868645804465885, 0.0001868645804465885, -0.0001868645804465885, 0.0001868645804465885, -0.0001868645804465885, 0.0001868645804465885]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=False,done_sucess=False
Reward for getting closser
Entered step
Unpause sim
Set action
Action:
[ 2.552151   -0.88584435 -0.35636094  2.7275398   3.0321205   2.1326532 ]
2.13265323638916
Error setting joint target. Is th

[ WARN] [1631016627.452219718, 157675.713000000]: Fail: ABORTED: No motion plan found. No execution attempted.


Set grip angle
[-0.0001861648124039661, 0.0001861648124039661, -0.0001861648124039661, 0.0001861648124039661, -0.0001861648124039661, 0.0001861648124039661]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=False,done_sucess=False
Reward for getting closser
Entered step
Unpause sim
Set action
Action:
[ 1.5855898   1.2547202   0.5840178   2.7675328   0.01696636 -1.2529    ]
-1.2529000043869019
Set grip angle
[-0.00018660549298665785, 0.00018660549298665785, -0.00018660549298665785, 0.00018660549298665785, -0.00018660549298665785, 0.00018660549298665785]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=False,done_sucess=False
Reward for getting closser
Entered step
Unpause sim
Set action
Action:
[-1.112088   -2.5191045  -3.022711    0.38953438  1.1646645   3.1123862 ]
3.1123862266540527
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.000186602390037649, 0.000186602390037649, -0.000186602390037649, 0.000186602390037649, -0.000186602390037649, 0.000186602390037649]
Get Obs
Is 

[ WARN] [1631016633.081438551, 157697.629000000]: Fail: ABORTED: No motion plan found. No execution attempted.


Entered step
Unpause sim
Set action
Action:
[-1.1672442  0.3157337  2.2229059  0.9666622  0.8241559 -1.9100088]
-1.9100087881088257
Set grip angle
[-0.0001859504372392351, 0.0001859504372392351, -0.0001859504372392351, 0.0001859504372392351, -0.0001859504372392351, 0.0001859504372392351]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=False,done_sucess=False
Reward for getting closser
Entered step
Unpause sim
Set action
Action:
[-0.12972595  2.4845285  -0.85070795  2.6102643  -0.8728752  -2.1199856 ]
-2.119985580444336
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.00018595043723834692, 0.00018595043723834692, -0.00018595043723834692, 0.00018595043723834692, -0.00018595043723834692, 0.00018595043723834692]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=True,done_sucess=False
Entered reset
Entered step
Unpause sim
Set action
Action:
[-1.5494589 -1.0637208  2.645771  -1.4935426  1.8805499 -2.2469199]
-2.246919870376587
Error setting joint target. Is the target within bo

[ WARN] [1631016637.055182934, 157714.043000000]: Fail: ABORTED: No motion plan found. No execution attempted.


Set grip angle
[-0.00018616499655887964, 0.00018616499655887964, -0.00018616499655887964, 0.00018616499655887964, -0.00018616499655887964, 0.00018616499655887964]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=False,done_sucess=False
Reward for getting closser
Entered step
Unpause sim
Set action
Action:
[-1.666873  -2.039886  -1.7080324  0.6515547  2.38665   -1.202643 ]
-1.2026430368423462
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.00018616499655887964, 0.00018616499655887964, -0.00018616499655887964, 0.00018616499655887964, -0.00018616499655887964, 0.00018616499655887964]
Get Obs
Is done
>>>>>>>>>>>>>>>>done_fail=True,done_sucess=False
Entered reset
Entered step
Unpause sim
Set action
Action:
[-2.4029353   1.4253913  -0.06606907  1.6924162   0.66170985  2.930597  ]
2.9305970668792725
Error setting joint target. Is the target within bounds?
Set grip angle
[-0.00018616499655976781, 0.00018616499655976781, -0.00018616499655976781, 0.00018616499655976781, -0.00

KeyboardInterrupt: 

In [ ]:
rb_observer.close()
reverb_server.stop()

In [ ]:
returns

In [ ]:
steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, episodes_lengths)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim()

In [ ]:
# from IPython.display import set_matplotlib_formats
# %matplotlib inline
# set_matplotlib_formats('svg')


# fig,ax = plt.subplots()
# plt.plot(steps,returns )
# plt.ylabel('Reward')
# plt.xlabel('Episode')
# plt.ylim()
# image_format = 'svg' # e.g .png, .svg, etc.
# image_name = 'avgReward_6800E_100S_RewardExplotation.svg'

# fig.savefig(image_name, format=image_format, dpi=1200)